In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
import food.custom_pandas as cpd
from tqdm import tqdm
import requests 

# !nbdev_build_lib

search best match of food classes to images (5)

- request glovo api for best match  
- create sql table corresponding photo_clips
- save clips to sql table

In [55]:
query = f"""select f.id, f.text, f.description from food.foods_prompted f
            left join food.foods_prompted_images i on (f.id = i.food_id)
            where i.clip is null"""

bs = 1


total = engine.execute(f'select count(*) from ({query}) a').one()

pd_iter = cpd.read_sql_query(query, engine, chunksize=1, index_col='id')

for df in tqdm(pd_iter, desc=f"clip image inference {schema}", total=total[0] // bs):  
    food_id = df.iloc[0].name
    text = df.iloc[0]['text']
    description = df.iloc[0]['description']

    url = f"http://localhost:8184/search"
    r_df = pd.read_json(requests.post(url,params={'text':text,'topk':5,'return_clip':True}).json())[['country_code','store_name','product_name','path','accuracy','clip']]
    r_df['food_id'] = food_id
    r_df.to_sql('foods_prompted_images', engine, if_exists='append',schema=schema,index=False)



Exception during reset or similar
Traceback (most recent call last):
  File "/home/dima/food/food/custom_pandas.py", line 14, in read_sql_query
    yield _wrap_result(partition,
GeneratorExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
psycopg2.errors.AdminShutdown: terminating connection due to administrator command
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

clip image inference food:  62%|█

KeyboardInterrupt: 